In [1]:
import sys
sys.path.append("/home/alex/Projects_py/scGPT")

In [2]:
import numpy as np
import torch
import scgpt as scg
from scgpt.model import TransformerGenerator
from tutorials.alex_perturbation._predict import plot_perturbation, predict
from tutorials.alex_perturbation._load_data import (
    _load_foundational_vocabulary_add_spec_tokens,
    _harmonize_pert_dataset_with_foundational_model,
    _load_perturbation_dataset
)
from tutorials.alex_perturbation._conf_perturb import (
    INPT_PAR,
    get_foundation_model_parameters,
    perts_to_plot
)
from _conf_perturb import (
    TRN_PAR, INPT_PAR, perturbation_data_source, split
)
from tutorials.alex_perturbation._conf_perturb import device
from scgpt.tokenizer.gene_tokenizer import GeneVocab
from _utils import get_perturb_folder, get_root_folder

/home/alex/anaconda3/envs/cuda_scgpt_310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
foundational_model_path = get_root_folder() / "save/scGPT_human"
model_config_file = foundational_model_path / "args.json"
found_model_file = foundational_model_path / "best_model.pt"
found_vocab_file = foundational_model_path / "vocab.json"

# model vocabulary:  60697, gene names: A1BG etc
vocab_foundational: GeneVocab = _load_foundational_vocabulary_add_spec_tokens(found_vocab_file)


In [4]:
pert_data = _load_perturbation_dataset(perturbation_data_source, split)
gene_ids: np.ndarray    # all gene names which are in pert dataset
n_genes_pert: int       # num of such genes
gene_ids, n_genes_pert, pert_data = _harmonize_pert_dataset_with_foundational_model(pert_data, vocab_foundational)


scGPT - INFO - Running on 2024-01-09 20:32:07


Found local copy...
Local copy of pyg dataset is detected. Loading...
Done!
Local copy of split is detected. Loading...
Simulation split test composition:
combo_seen0:0
combo_seen1:0
combo_seen2:0
unseen_single:22
Done!
Creating dataloaders....
Done!


scGPT - INFO - match 4399/5060 genes in vocabulary of size 60697.


In [18]:
obs = pert_data.adata.obs
obs.sort_index()

,condition,cell_type,dose_val,control,condition_name
cell_barcode,,,,,
AAACATACAACCGT-9,HSPA5+ctrl,K562(?),1+1,0,K562(?)_HSPA5+ctrl_1+1
AAACATACAAGATG-1,ctrl,K562(?),1,1,K562(?)_ctrl_1
AAACATACACACTG-2,ctrl,K562(?),1,1,K562(?)_ctrl_1
AAACATACACCGAT-1,CREB1+ctrl,K562(?),1+1,0,K562(?)_CREB1+ctrl_1+1
AAACATACACCTAG-1,OST4+ctrl,K562(?),1+1,0,K562(?)_OST4+ctrl_1+1
...,...,...,...,...,...
TTTGCATGTTCCAT-8,ctrl,K562(?),1,1,K562(?)_ctrl_1
TTTGCATGTTCTAC-1,ctrl,K562(?),1,1,K562(?)_ctrl_1
TTTGCATGTTCTGT-1,ctrl,K562(?),1,1,K562(?)_ctrl_1


In [8]:
obs.shape

(68603, 5)

In [10]:
var = pert_data.adata.var
var

,gene_name,id_in_vocab
ENSG00000228463,AP006222.2,-1
ENSG00000272438,RP11-54O7.16,1
ENSG00000230699,RP11-54O7.1,1
ENSG00000223764,RP11-54O7.3,-1
ENSG00000187634,SAMD11,1
...,...,...
ENSG00000233922,AL133493.2,-1
ENSG00000142156,COL6A1,1
ENSG00000228137,AP001469.7,1
ENSG00000160298,C21orf58,1


In [13]:
x = pert_data.adata.X
type(x)

scipy.sparse._csr.csr_matrix

In [17]:
x[1:10, 1:10].toarray()

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.7198251 , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.84545773, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.9277227 ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        1.3558942 , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.882075  , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 1.4457473 ,
        0.9641325 , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
